In [ ]:
%load_ext tensorboard

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pano_generator
import os

c:\Users\ual\.conda\envs\felix\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
os.listdir("training_results/last_model/B0_All")

[]

In [8]:
model = tf.keras.models.load_model("training_results/last_model/B0_All/")

OSError: SavedModel file does not exist at: training_results/last_model/B0_All/\{saved_model.pbtxt|saved_model.pb}

In [6]:
eval("tf.keras.applications.MobileNetV3Large")

<function keras.applications.mobilenet_v3.MobileNetV3Large(input_shape=None, alpha=1.0, minimalistic=False, include_top=True, weights='imagenet', input_tensor=None, classes=1000, pooling=None, dropout_rate=0.2, classifier_activation='softmax', include_preprocessing=True)>

In [2]:
import tensorflow.keras.layers as layers

# some img_augmentation
img_augmentation = tf.keras.models.Sequential(
    [
        tf.keras.layers.RandomRotation(factor=0.15),
        tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        tf.keras.layers.RandomFlip(mode="horizontal"),
        tf.keras.layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def transfer_model(number_classes,architecture = tf.keras.applications.efficientnet.EfficientNetB2,name=None):

    inputs = tf.keras.Input([260,260,3])

    x = img_augmentation(inputs) # apply image augmentaion

    model = architecture(
        include_top=False,
        weights='imagenet',
        input_tensor= x)

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(number_classes, activation="softmax", name="pred",kernel_regularizer=tf.keras.regularizers.L2())(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name=name)

    return model


def unfreeze_model(model,n_layers = 20):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-n_layers:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True


    return model

In [3]:
rescale = False
inputs = tf.keras.Input([260,260,3])

x = img_augmentation(inputs) # apply image augmentaion

if rescale:
    x = tf.keras.layers.Resizing(224,224) (x)

model =  tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_tensor= x)

# Freeze the pretrained weights
model.trainable = False

# Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(2, activation="softmax", name="pred",kernel_regularizer=tf.keras.regularizers.L2())(x)

# Compile
model = tf.keras.Model(inputs, outputs, name="b0_test")



In [4]:
model.compile(tf.keras.optimizers.Adam(1e-4),loss = tf.keras.losses.BinaryCrossentropy(),metrics=["accuracy"])

In [5]:
log_dir =  "training_results/logs/" + model.name + "/"
checkpoint_path =  "training_results/weights/" + model.name + "/"
last_model = "training_results/last_model/" + model.name + "/"

try:
    os.makedirs(log_dir)
    os.makedirs(checkpoint_path)
    os.makedirs(last_model)
except:
    pass


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor='val_accuracy',
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    mode='auto') 

In [6]:
train_ds , val_ds = pano_generator.get_train_and_val()

if rescale:
    train_ds = train_ds.map(lambda x,y : (x/255-1,y))
    val_ds = val_ds.map(lambda x,y : (x/255-1,y))

In [7]:
with tf.device('/device:GPU:0'):
    model.fit(train_ds,epochs=20,validation_data=val_ds,callbacks=[tensorboard_callback,cp_callback,early_stopping_callback])

Epoch 1/20
515/515 [==============================] - ETA: 0s - loss: 0.2715 - accuracy: 0.9280
Epoch 1: val_accuracy improved from -inf to 0.86155, saving model to training_results/weights/b0_test\
515/515 [==============================] - 377s 714ms/step - loss: 0.2715 - accuracy: 0.9280 - val_loss: 0.3303 - val_accuracy: 0.8615
Epoch 2/20
515/515 [==============================] - ETA: 0s - loss: 0.0976 - accuracy: 0.9897
Epoch 2: val_accuracy improved from 0.86155 to 0.87153, saving model to training_results/weights/b0_test\
515/515 [==============================] - 371s 721ms/step - loss: 0.0976 - accuracy: 0.9897 - val_loss: 0.3117 - val_accuracy: 0.8715
Epoch 3/20
515/515 [==============================] - ETA: 0s - loss: 0.0769 - accuracy: 0.9927
Epoch 3: val_accuracy improved from 0.87153 to 0.88455, saving model to training_results/weights/b0_test\
515/515 [==============================] - 373s 725ms/step - loss: 0.0769 - accuracy: 0.9927 - val_loss: 0.2780 - val_accuracy:

KeyboardInterrupt: 

In [8]:
# save last model
model.save(last_model+"last_model")
# load best weights
model.load_weights(checkpoint_path)

INFO:tensorflow:Assets written to: training_results/last_model/b0_test/last_model\assets


INFO:tensorflow:Assets written to: training_results/last_model/b0_test/last_model\assets


In [10]:
model = unfreeze_model(model,n_layers = 20)
model.compile(tf.keras.optimizers.Adam(1e-4),loss = tf.keras.losses.BinaryCrossentropy(),metrics=["accuracy"])

In [11]:
model._name = "B0_All"
log_dir =  "training_results/logs/" + model.name + "/"
checkpoint_path =  "training_results/weights/" + model.name + "/"
last_model = "training_results/last_model/" + model.name + "/"

try:
    os.makedirs(log_dir)
    os.makedirs(checkpoint_path)
    os.makedirs(last_model)
except:
    pass

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor='val_accuracy',
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    verbose=1,
    mode='auto') 

model.fit(train_ds,epochs=60,validation_data=val_ds,callbacks=[tensorboard_callback,cp_callback,early_stopping_callback , reduce_lr])


Epoch 1/60
